In [1]:
import gym
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from tqdm import tqdm
import imageio

In [33]:
env = gym.make("ALE/BattleZone-v5", render_mode="rgb_array")

print("Action Space:", env.action_space)
print("Observation Space:", env.observation_space)

class DQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(DQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        self.fc = nn.Sequential(
            nn.Linear(self._calculate_conv_output(input_shape), 512),
            nn.ReLU(),
            nn.Linear(512, num_actions)
        )
    
    def _calculate_conv_output(self, shape):
        with torch.no_grad():
            o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
    
    def forward(self, x):
        x = self.conv(x)
        x = x.reshape(x.size(0), -1)
        return self.fc(x)

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.array(state), action, reward, np.array(next_state), done
    
    def __len__(self):
        return len(self.buffer)

class DQNAgent:
    def __init__(self, input_shape, num_actions):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.num_actions = num_actions
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.99995
        self.gamma = 0.95
        self.learning_rate = 0.0001
        self.batch_size = 32
        self.memory_capacity = 30000
        
        self.policy_net = DQN(input_shape, num_actions).to(self.device)
        self.target_net = DQN(input_shape, num_actions).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
        self.memory = ReplayBuffer(self.memory_capacity)
        
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        self.loss_fn = nn.MSELoss()
        
    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.num_actions - 1)
        else:
            state = np.transpose(state, (2, 0, 1))  # Convert (H, W, C) -> (C, H, W)
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
            q_values = self.policy_net(state)
            return q_values.max(1)[1].item()

    def update(self):
        if len(self.memory) < self.batch_size:
            return

        states, actions, rewards, next_states, dones = self.memory.sample(self.batch_size)
        states = np.transpose(np.array(states), (0, 3, 1, 2))
        next_states = np.transpose(np.array(next_states), (0, 3, 1, 2))

        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions).unsqueeze(1).to(self.device)
        rewards = torch.tensor(rewards).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).to(self.device)

        q_values = self.policy_net(states).gather(1, actions)
        next_q_values = self.target_net(next_states).max(1)[0].detach()
        target_q_values = rewards + (self.gamma * next_q_values * (1 - dones))

        loss = self.loss_fn(q_values.squeeze(), target_q_values)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        rewards_history.append(total_reward)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    
    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

num_episodes = 47
max_steps_per_episode = 10000
input_shape = (3, 210, 160)
num_actions = env.action_space.n
rewards_history = []

agent = DQNAgent(input_shape, num_actions)

for episode in tqdm(range(num_episodes), desc="Training Progress"):
    state, _ = env.reset()
    total_reward = 0
    
    for step in range(max_steps_per_episode):
        action = agent.select_action(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        agent.memory.push(state, action, reward, next_state, done)
        agent.update()
        
        state = next_state
        total_reward += reward
        
        if done:
            break
    
    if episode % 5 == 0:
        agent.update_target_network()
    
    print(f"Episode {episode+1}/{num_episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.4f}")

env.close()

torch.save(agent.policy_net.state_dict(), "dqn_policy_weights.pth")
torch.save(agent.target_net.state_dict(), "dqn_target_weights.pth")

with open("rewards_history.json", "w") as f:
    json.dump(rewards_history, f)

def record_video(env, agent, out_path, fps=30):
    frames = []
    state, _ = env.reset()
    done = False
    while not done:
        frame = env.render()
        frames.append(frame)
        
        action = agent.select_action(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        state = next_state
    
    imageio.mimsave(out_path, frames, fps=fps)

record_video(env, agent, "/Users/romain/Rein/battlezone_play.mp4")


Action Space: Discrete(18)
Observation Space: Box(0, 255, (210, 160, 3), uint8)


Training Progress:   2%|▌                        | 1/47 [01:08<52:08, 68.01s/it]

Episode 1/47, Total Reward: 5000.0, Epsilon: 0.9326


Training Progress:   4%|█                        | 2/47 [01:55<41:53, 55.85s/it]

Episode 2/47, Total Reward: 2000.0, Epsilon: 0.8886


...

Episode 45/47, Total Reward: 2000.0, Epsilon: 0.1000


Training Progress:  98%|███████████████████████▍| 46/47 [55:41<01:28, 88.17s/it]

Episode 46/47, Total Reward: 5000.0, Epsilon: 0.1000


Training Progress: 100%|████████████████████████| 47/47 [56:25<00:00, 72.03s/it]

Episode 47/47, Total Reward: 2000.0, Epsilon: 0.1000


In [35]:
with open("rewards_history.json", "w") as f:
    json.dump(rewards_history, f)
torch.save(agent.policy_net.state_dict(), "dqn_policy_weights.pth")
torch.save(agent.target_net.state_dict(), "dqn_target_weights.pth")

In [41]:
policy_weights = torch.load("dqn_policy_weights.pth")

for name, weight in policy_weights.items():
    print(f"{name}: {weight.shape}")
    print(weight)
    break

conv.0.weight: torch.Size([32, 3, 8, 8])
tensor([[[[ 0.0166, -0.0155,  0.0329,  ..., -0.0142, -0.0528,  0.0652],
          [-0.0304, -0.0211,  0.0529,  ..., -0.0389,  0.0263,  0.0414],
          [-0.0318, -0.0312, -0.0301,  ..., -0.1082, -0.1115, -0.0698],
          ...,
          [-0.0299,  0.0591,  0.0680,  ..., -0.0145, -0.0033,  0.0792],
          [-0.0555, -0.0620,  0.0859,  ..., -0.0184, -0.1053,  0.0798],
          [-0.0714, -0.1015,  0.0053,  ..., -0.0668, -0.0589, -0.0394]],

         [[-0.0657,  0.0215,  0.0217,  ...,  0.0607,  0.0148, -0.0263],
          [ 0.0151,  0.0118, -0.0085,  ..., -0.0392, -0.0620,  0.0486],
          [ 0.0149,  0.0315,  0.1018,  ...,  0.0433, -0.0978, -0.0556],
          ...,
          [ 0.0769,  0.0185,  0.0964,  ...,  0.0683, -0.0710,  0.0325],
          [ 0.0281,  0.0636,  0.0039,  ...,  0.0331, -0.0523,  0.0727],
          [-0.0264, -0.0233,  0.0522,  ...,  0.0415,  0.0164, -0.0671]],

         [[-0.0321,  0.0774,  0.0380,  ..., -0.0131,  0.0013,

In [49]:
import torch

# Задаём параметры модели
input_shape = (3, 210, 160)  # Размер входных данных (например, для RGB-изображения)
num_actions = 18  # Количество возможных действий (зависит от вашей среды)

# Загружаем модель (предположим, что DQN определена в вашем коде)
model = DQN(input_shape, num_actions)

# Загружаем веса в модель
model.load_state_dict(torch.load('dqn_policy_weights.pth'))
model.eval()  # Устанавливаем модель в режим оценки (выключает dropout и batch normalization)
with torch.no_grad():  # Отключаем градиенты, так как они нам не нужны для предсказаний
    q_values = model(random_input_tensor)  # Получаем Q-значения для всех возможных действий

# Выводим результат
print("Q-values for the input:", q_values)

# Получаем индекс действия с максимальным Q-значением
best_action = q_values.argmax().item()
print(f"Best action based on the Q-values: {best_action}")


Q-values for the input: tensor([[0.2117, 0.2958, 0.1526, 0.1713, 0.3313, 0.2374, 0.2093, 0.2602, 0.1805,
         0.2498, 0.1894, 0.2541, 0.2192, 0.3707, 0.4587, 0.3392, 0.2776, 0.4882]])
Best action based on the Q-values: 17
